In [15]:
import pickle
import pandas as pd
from connections import AWS

$\textbf{Epidemiology: Clinical Model Training}$

Injury risk estimation is possible from two perspectives: 
- __Season__ (i.e., post-outing injury probability; postgame)
- __Pitch-Level__ (i.e., next-pitch injury probability; within game)

This notebook uses the data structures and model architecture established in `clinical_model_setup.ipynb`. The architecture was shown to work with a manually crafted batched-training loop.

In [2]:
# setup AWS connection
aws = AWS()
aws.connect()

[AWS]: Port 5433 is free.
[AWS]: Connected to RDS endpoint.


In [ ]:
# download most up-to-date tensor dictionaries
pitch_level_tensors = aws.s3.download_file(
    aws.bucket_name, 
    'epidemiology/ml/datasets/pytorch/pitch_level_tensors.pkl', 
    'storage/pitch_level_tensors.pkl'
)

# load tensors into memory
with open('storage/pitch_level_tensors.pkl', 'rb') as f:
    pitch_level_tensors = pickle.load(f)

$\textbf{Model Development}$

In [123]:
import torch
from tqdm import tqdm
import torch.nn as nn
from nnet import CNNbiLSTM
import torch.nn.functional as F
from nnet.loss_functions import pitch_level_loss
from sklearn.metrics import roc_auc_score, f1_score
from services.scalers import compute_masked_scalers, apply_scalers

In [124]:
# compile model for training --> model, optimizer, loss function, and data setup
def compile_model(
        train_data: dict,
        val_data: dict,
        model_config: dict = {
            'stem': 64,
            'c': 96,
            'kernel': 7,
            'lstm_hidden': 128,
            'dropout': 0.1,
            'bidir': True
        },
        device: str = 'cpu',
        use_pos_weight: bool = False
) -> dict:
    """ 
    Compile a nnet architecture for training. Includes scaling, model instantiation, and loss function setup.
    
    Args:
        train_data (dict): Training data containing tensors and masks.
        val_data (dict): Validation data containing tensors and masks.
        model_config (dict): Configuration for the CNNbiLSTM model. Defaults to basic setup.
        device (str): Device to run the model on ('cpu' or 'cuda').
        use_pos_weight (bool): Whether to use positive class weights in loss function.
    
    Returns:
        model_setup (dict): Dictionary with all relevant information..
    """

    # standardize example sequence --> example training tensor sequence (x)
    mean, std = compute_masked_scalers(train_data['seq'], train_data['mask'])
    x = apply_scalers(train_data['seq'], mean, std)
    B, T, K = x.shape       # NOTE: B = batch size, T = time steps, K = features; K is used for model setup

    # setup (device, shapes)
        # --> move full tensors to device
    device = "cuda" if torch.cuda.is_available() else "cpu"
    x_trn, y_step_trn, y_binary_trn, mask_trn, lengths_trn = move_tensors_to_device(x, train_data, device)

    # setup model, optimizer
    model = CNNbiLSTM(
        k_in=K, 
        stem=model_config.get('stem', 64),
        c=model_config.get('c', 96),
        kernel=model_config.get('kernel', 7),
        lstm_hidden=model_config.get('lstm_hidden', 128),
        dropout=model_config.get('dropout', 0.1),
        bidir=model_config.get('bidir', True),
    ).to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)

    # optional: compute pos_weight over valid steps once
        # NOTE: for probs this should be 1.0
    if use_pos_weight:
        with torch.no_grad():
            pos = y_step_trn[mask_trn].sum()
            tot = mask_trn.sum()
            neg = tot - pos
            pos_weight = (neg / pos.clamp(min=1)).float()
    else:
        pos_weight = torch.tensor(1.0, device=device)

    # package all into dictionary
    model_setup = {
        'model': model.train(),
        'optimizer': optimizer,
        'loss_fn': pitch_level_loss,
        'pos_weight': pos_weight,
        'mean': mean,
        'std': std,
        'device': device,
        'train_setup': {
            'x': x_trn,
            'y_step': y_step_trn,
            'y_binary': y_binary_trn,
            'mask': mask_trn,
            'lengths': lengths_trn
        },
        'val_setup': {
            'x': apply_scalers(val_data['seq'], mean, std).to(device),
            'y_step': val_data['probs'].float().to(device),
            'y_binary': val_data['binary'].float().to(device),
            'mask': val_data['mask'].bool().to(device),
            'lengths': val_data['lengths'].float().to(device)
        }
    }

    return model_setup

# move tensors to device
    # mostly a helper function for training data 
def move_tensors_to_device(
        x: torch.Tensor,
        train_data: dict,
        device: str
) -> tuple[torch.Tensor, torch.Tensor, torch.Tensor, torch.Tensor]:
    """
    Move tensors to the specified device.
    
    Args:
        x (torch.Tensor): Input tensor.
        train_data (dict): Dictionary containing training data tensors.
        device (str): Device to move tensors to ('cpu' or 'cuda').
    
    Returns:
        tuple: Tensors moved to the specified device.
    """
    return x.float().to(device), train_data['probs'].float().to(device), train_data['binary'].float().to(device), train_data['mask'].bool().to(device), train_data['lengths'].long().to(device)
    
# early stopping check
def check_early_stopping(
        val_loss: float,
        best_val_loss: float,
        epochs_no_improve: int,
        patience: int
) -> tuple[bool, float, int]:
    """
    Check if early stopping criteria are met.
    
    Args:
        val_loss (float): Current validation loss.
        best_val_loss (float): Best validation loss observed so far.
        epochs_no_improve (int): Number of epochs since last improvement.
        patience (int): Patience for early stopping.
    
    Returns:
        tuple: (should_stop (bool), best_val_loss (float), epochs_no_improve (int))
    """
    if val_loss < best_val_loss:
        return False, val_loss, 0
    else:
        epochs_no_improve += 1
        if epochs_no_improve >= patience:
            return True, best_val_loss, epochs_no_improve
        else:
            return False, best_val_loss, epochs_no_improve

# ROC/AUC calcs
def compute_roc_auc( 
        probs: torch.Tensor,
        y_true: torch.Tensor,
        mask: torch.Tensor
) -> tuple[float, float, float]:
    """
    Compute ROC AUC score.
    
    Args:
        probs (torch.Tensor): Predicted probabilities.
        y_true (torch.Tensor): True labels.
        mask (torch.Tensor): Mask indicating valid entries.
    
    Returns:
        tuple: (batch_scores (torch.Tensor), batch_labels (torch.Tensor), roc_auc (float))
    """
    # per-batch valid scores/labels for AUC (use binary labels)
    batch_scores = probs.detach().float()
    batch_labels = y_true[mask].detach().float()

    # compute AUC
    if batch_labels.sum() == 0 or batch_labels.sum() == len(batch_labels):
        roc_auc = 0.5 
    else:
        roc_auc = roc_auc_score(batch_labels.cpu().numpy(), batch_scores.cpu().numpy())

    return batch_scores, batch_labels, roc_auc

# training loop for a model
    # NOTE: sets up a dictionary for storing model loss history
def train_model(
        model_name: str,
        model_setup: dict,
        n_epochs: int = 100,
        batch_size: int = 32,
        patience: int = 10,
        verbose: bool = True
) -> dict:
    """
    Train the model using the provided training and validation data. Manually handles batching and early stopping.
    
    Args:
        model_setup (dict): Dictionary containing model, optimizer, loss function, and data setup.
        n_epochs (int): Number of epochs to train. Defaults to 100.
        batch_size (int): Batch size for training. Defaults to 32.
        patience (int): Patience for early stopping. Defaults to 10.
        verbose (bool): Whether to print training progress. Defaults to True.
    
    Returns:
        dict: Dictionary containing training and validation loss history.
    """
    # unpack model setup
    model = model_setup['model']
    optimizer = model_setup['optimizer']
    loss_fn = model_setup['loss_fn']
    pos_weight = model_setup['pos_weight']

    # extract training data
    x = model_setup['train_setup']['x']
    y_step = model_setup['train_setup']['y_step']
    y_step_binary = model_setup['train_setup']['y_binary']
    mask = model_setup['train_setup']['mask']
    lengths = model_setup['train_setup']['lengths']

    # setup loss storage
    history = {
        'train_loss': [],
        'train_acc': [],
        'train_auc': [],
        'train_f1': [],
        'val_loss': [],
        'val_acc': [],
        'val_auc': [],
        'val_f1': [],
        'best_val_loss': None,
        'stopped_epoch': None
    }

    # early stopping setup
    best_val_loss = float('inf')
    epochs_no_improve = 0

    # AUC setup
    epoch_y_true = []
    epoch_y_score = []

    # F1 setup
    epoch_preds = []
    epoch_labels = []

    # MAIN LOOP: iterate through epochs
    B = x.shape[0]
    for epoch in range(1, n_epochs+1):
        
        """ TRAINING """
        # set model to train mode, shuffle indices, create batches
        model.train()
        idx = torch.randperm(B, device=model_setup['device'])  # shuffle indices each epoch

        # loss counters
        running_loss = 0.0
        running_correct = 0
        running_total = 0

        # wrap range() with tqdm
        pbar = tqdm(range(0, B, batch_size), desc=f"Epoch {epoch}/{n_epochs}", leave=False)
        for start in pbar:
            end = min(start + batch_size, B)
            bidx = idx[start:end]

            # forward pass
            xb = x[bidx]
            yb = y_step[bidx]
            mb = mask[bidx]
            Lb = lengths[bidx]

            # update loss
            logits = model(xb, Lb)
            loss = F.binary_cross_entropy_with_logits(
                logits[mb], yb[mb], pos_weight=pos_weight
            )

            # back propagation
            optimizer.zero_grad(set_to_none=True)
            loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()

            # running loss
            bs = xb.size(0)
            running_loss += loss.item() * bs
            
            # running accuracy
            probs  = torch.sigmoid(logits[mb])
            preds  = (probs > 0.5).float()
            correct = (preds == y_step_binary[bidx][mb]).sum().item()
            total   = mb.sum().item()

            # update epoch AUC
            batch_scores, batch_labels, roc_auc = compute_roc_auc(probs, y_step_binary[bidx], mb)
            epoch_y_true.extend(batch_labels.cpu().numpy())
            epoch_y_score.extend(batch_scores.cpu().numpy())

            # update total
            running_correct += correct
            running_total += total

            # update accuracy
            run_avg_loss = running_loss / ((start // batch_size + 1) * bs)
            run_acc      = running_correct / running_total

            # gather & store F1 data
            batch_preds = preds.detach().cpu()
            batch_labels = y_step_binary[bidx][mb].detach().cpu()
            epoch_preds.append(batch_preds)
            epoch_labels.append(batch_labels)

            # show both current and running losses in the bar
            pbar.set_postfix(loss=f"{run_avg_loss:.4f}", acc=f"{run_acc:.4f}", auc=f"{roc_auc:.4f}")

        # update epoch loss
        epoch_loss = running_loss / B
        epoch_acc  = running_correct / running_total
        epoch_auc = roc_auc_score(epoch_y_true, epoch_y_score)

        # update F1 score
        epoch_preds_cat = torch.cat(epoch_preds).numpy()
        epoch_labels_cat = torch.cat(epoch_labels).numpy()
        epoch_f1 = f1_score(epoch_labels_cat, epoch_preds_cat)
        
        # print epoch update
        if verbose:
            print(f"Epoch {epoch:02d} | Training Loss: {epoch_loss:.3f} | Training Accuracy: {epoch_acc:.3f} | Training AUC: {epoch_auc:.3f} | Training F1: {epoch_f1:.3f}")

        # update history
        history['train_loss'].append(epoch_loss)
        history['train_acc'].append(epoch_acc)
        history['train_auc'].append(epoch_auc)
        history['train_f1'].append(epoch_f1)

        """ VALIDATION """
        # set model to eval mode
        model.eval()
        with torch.no_grad():
            logits = model(
                model_setup['val_setup']['x'].float(), 
                model_setup['val_setup']['lengths'].float()
            )
            val_loss = loss_fn(
                logits,
                model_setup['val_setup']['y_step'],
                model_setup['val_setup']['mask'],
                pos_weight
            ).item()

            # compute accuracy
            probs  = torch.sigmoid(logits[model_setup['val_setup']['mask']])
            preds  = (probs > 0.5).float()
            ground_truth = model_setup['val_setup']['y_binary'][model_setup['val_setup']['mask']]
            correct = (preds == ground_truth).sum().item()
            total   = model_setup['val_setup']['mask'].sum().item()
            val_acc = correct / total

            # compute AUC, F1
            val_auc = roc_auc_score(ground_truth.numpy(), probs.numpy())
            val_f1 = f1_score(ground_truth.numpy(), preds.numpy())

            # print update
            if verbose:
                print(f"Epoch {epoch:02d} | Validation Loss: {val_loss:.3f} | Validation Accuracy: {val_acc:.3f} | Validation AUC: {val_auc:.3f} | Validation F1: {val_f1:.3f}")

            # update history
            history['val_loss'].append(val_loss)
            history['val_acc'].append(val_acc)
            history['val_auc'].append(val_auc)
            history['val_f1'].append(val_f1)

            # check for improvement
            stop, best_val_loss, epochs_no_improve = check_early_stopping(
                val_loss, best_val_loss, epochs_no_improve, patience
            )
            if stop:
                print(f"Early stopping triggered after {epoch} epochs.")
                
                # update history
                history['best_val_loss'] = best_val_loss
                history['stopped_epoch'] = epoch

                # save model (+ state dict)
                    # ensure model is in eval mode before saving
                model.eval()
                torch.save(model, f"models/inj/{model_name}.pt")
                torch.save(model.state_dict(), f"models/inj/{model_name}_state_dict.pt")
                
                return history  # exit training loop
            
            else:
                # update best val loss in history
                history['best_val_loss'] = best_val_loss

    # if we reach here, training completed without early stopping
    if verbose:
        print(f"Training completed after {epoch} epochs with best validation loss: {best_val_loss:.3f}")

    # save final model
        # ensure model is in eval mode before saving
    model.eval()
    torch.save(model, f"models/inj/{model_name}.pt")
    torch.save(model.state_dict(), f"models/inj/{model_name}_state_dict.pt")

    return history


In [151]:
# create model setups for outing- and pitch-level data by day windows
model_setups = {}
model_results = {}

# compile models for each day window
for day in [7, 15, 30, 45, 90]:
    model_setups[day] = compile_model(
        train_data=pitch_level_tensors['trn'][day],
        val_data=pitch_level_tensors['val'][day]
    )
    

In [152]:
""" PITCH-LEVEL MODELS """
for day in [7, 15, 30, 45, 90]:
    
    print(f'Training pitch-level model for preceding day window: {day} days')
    
    # pitch-level model loop
    pitch_model_results = train_model(
        model_name=f'pitch_model_{day}d',
        model_setup=model_setups[day],
        n_epochs=100,
        batch_size=32,
        patience=5,
        verbose=True
    )

    # save to disk, upload to S3
    with open(f'storage/pitch_model_{day}d_results.pkl', 'wb') as f:
        pickle.dump(pitch_model_results, f)
    with open(f'storage/pitch_model_{day}d_results.pkl', 'rb') as f:
        content = f.read()
        aws.upload_to_s3(content, f'epidemiology/ml/models/inj/pitch_model_{day}d_results.pkl')

Training pitch-level model for preceding day window: 7 days


Epoch 01 | Training Loss: 0.804 | Training Accuracy: 0.522 | Training AUC: 0.545 | Training F1: 0.555
Epoch 01 | Validation Loss: 0.677 | Validation Accuracy: 0.556 | Validation AUC: 0.662 | Validation F1: 0.225


Epoch 02 | Training Loss: 0.655 | Training Accuracy: 0.570 | Training AUC: 0.571 | Training F1: 0.558
Epoch 02 | Validation Loss: 0.618 | Validation Accuracy: 0.601 | Validation AUC: 0.693 | Validation F1: 0.469


Epoch 03 | Training Loss: 0.623 | Training Accuracy: 0.635 | Training AUC: 0.617 | Training F1: 0.587
Epoch 03 | Validation Loss: 0.486 | Validation Accuracy: 0.782 | Validation AUC: 0.855 | Validation F1: 0.737


Epoch 04 | Training Loss: 0.476 | Training Accuracy: 0.782 | Training AUC: 0.688 | Training F1: 0.628
Epoch 04 | Validation Loss: 0.417 | Validation Accuracy: 0.836 | Validation AUC: 0.931 | Validation F1: 0.819


...

Epoch 46 | Training Loss: 0.283 | Training Accuracy: 0.964 | Training AUC: 0.935 | Training F1: 0.846
Epoch 46 | Validation Loss: 0.284 | Validation Accuracy: 0.962 | Validation AUC: 0.998 | Validation F1: 0.963


Epoch 47 | Training Loss: 0.285 | Training Accuracy: 0.961 | Training AUC: 0.936 | Training F1: 0.849
Epoch 47 | Validation Loss: 0.286 | Validation Accuracy: 0.960 | Validation AUC: 0.995 | Validation F1: 0.960


Epoch 48 | Training Loss: 0.283 | Training Accuracy: 0.968 | Training AUC: 0.938 | Training F1: 0.851
Epoch 48 | Validation Loss: 0.282 | Validation Accuracy: 0.970 | Validation AUC: 0.997 | Validation F1: 0.970


Epoch 49 | Training Loss: 0.280 | Training Accuracy: 0.972 | Training AUC: 0.940 | Training F1: 0.854
Epoch 49 | Validation Loss: 0.290 | Validation Accuracy: 0.961 | Validation AUC: 0.993 | Validation F1: 0.960


Epoch 50 | Training Loss: 0.285 | Training Accuracy: 0.961 | Training AUC: 0.941 | Training F1: 0.856
Epoch 50 | Validation Loss: 0.287 | Validation Accuracy: 0.954 | Validation AUC: 0.996 | Validation F1: 0.955
Early stopping triggered after 50 epochs.
[AWS]: Uploaded object to s3://pitch-ml/epidemiology/ml/models/inj/pitch_model_90d_results.pkl


$\textbf{Results Aggregation}$

Organize model results into a table:

- __Window__ (i.e., max. number of days to use)
- __Epochs Trained__ (prior to stopping)
- __Validation Scores__ (Accuracy, F1, ROC AUC)

In [134]:
with open('storage/pitch_model_results.pkl', 'rb') as f:
    model_results_loaded = pickle.load(f)

In [135]:
model_results_loaded

{}

$\textbf{Close AWS Connection}$

In [88]:
# close connection
aws.close()

[AWS]: Database connection closed.
[AWS]: SSH tunnel stopped.
